# DSC 80: Lab 07

### Due Date: Tuesday, May 18th 11:59PM

## Instructions
Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding work will be developed in an accompanying `lab*.py` file, that will be imported into the current notebook.

Labs and programming assignments will be graded in (at most) two ways:
1. The functions and classes in the accompanying python file will be tested (a la DSC 20),
2. The notebook will be graded (for graphs and free response questions).

**Do not change the function names in the `*.py` file**
- The functions in the `*.py` file are how your assignment is graded, and they are graded by their name. The dictionary at the end of the file (`GRADED FUNCTIONS`) contains the "grading list". The final function in the file allows your doctests to check that all the necessary functions exist.
- If you changed something you weren't supposed to, just use git to revert!

**Tips for working in the Notebook**:
- The notebooks serve to present you the questions and give you a place to present your results for later review.
- The notebook on *lab assignments* are not graded (only the `.py` file).
- Notebooks for PAs will serve as a final report for the assignment, and contain conclusions and answers to open ended questions that are graded.
- The notebook serves as a nice environment for 'pre-development' and experimentation before designing your function in your `.py` file.

**Tips for developing in the .py file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional functions to solve the lab! 
    - Developing in python usually consists of larger files, with many short functions.
    - You may write your other functions in an additional `.py` file that you import in `lab**.py` (much like we do in the notebook).
- Always document your code!

### Importing code from `lab**.py`

* We import our `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab**.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab**.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab**.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab**` merely import the existing compiled python.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lab07 as lab

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import time
import re

In [4]:
import requests
import json

# Practice with regular expressions (Regex)

**Question 1**

You start with some basic regular expression exercises to get some practice using them. You will find function stubs and related doctests in the starter code. 

**Exercise 1:** A string that has a `[` as the third character and `]` as the sixth character.

**Exercise 2:** Phone numbers that start with '(858)' and follow the format '(xxx) xxx-xxxx' (x represents a digit).

*Notice: There is a space between (xxx) and xxx-xxxx*

**Exercise 3:** A string whose length is between 6 to 10 and contains only word characters, white spaces and `?`. This string must have `?` as its last character.

**Exercise 4:** A string that begins with '\\$' and with another '\\$' within, where:
   - Characters between the two '\\$' can be anything (including nothing) except the letters 'a', 'b', 'c' (lower case).
   - Characters after the second '\\$' can only have any number of the letters 'a', 'b', 'c' (upper or lower case), with every 'a' before every 'b', and every 'b' before every 'c'.
       - E.g. 'AaBbbC' works, 'ACB' doesn't.

**Exercise 5:** A string that represents a valid Python file name including the extension. 

*Notice*: For simplicity, assume that the file name contains only letters, numbers and an underscore `_`.

**Exercise 6:** Find patterns of lowercase letters joined with an underscore.

**Exercise 7:** Find patterns that start with and end with a `_`.

**Exercise 8:**  Apple registration numbers and Apple hardware product serial numbers might have the number '0' (zero), but never the letter 'O'. Serial numbers don't have the number '1' (one) or the letter 'i'. Write a line of regex expression that checks if the given Serial number belongs to a genuine Apple product.

**Exercise 9:** Check if a given ID number is from Los Angeles (LAX), San Diego(SAN) or the state of New York (NY). ID numbers have the following format `SC-NN-CCC-NNNN`. 
   - SC represents state code in uppercase 
   - NN represents a number with 2 digits 
   - CCC represents a three letter city code in uppercase
   - NNNN represents a number with 4 digits

**Exercise 10:**  Given an input string, cast it to lower case, remove spaces/punctuation, and return a list of every 3-character substring following this logic:
   - The first character doesn't start with 'a' or 'A'
   - The last substring (and only the last substring) can be shorter than 3 characters, depending on the length of the input string.
   - The substrings cannot overlap
   
Here's an example with one of the doctests:

`>>> match_10("Ab..DEF")`
`['def']`

1. convert it to a lowercase string resulting in "ab..def"
2. delete any 3 letter sequence that starts with the letter 'a', so delete "ab." from the string, leaving using with ".def"
3. delete the punctuation resulting in "def"
4. finally, we get `["def"]`

(Only split in the last step, everything else is removing from the string)

In [258]:
s = ' DEFaabc !g '
s = s.lower().replace(' ', '')

s = re.findall('[^a]..|[\w\s]{3}', s)
s = list(map(lambda string: re.sub('[^\w\s]', '', string) , s))
s = list(filter(lambda x: '' if x[0] == 'a' else x, s))

In [209]:
re.sub('[^\w\s]', '', s[1])

'cg'

## Regex groups: extracting personal information from messy data

**Question 2**

The file in `data/messy.txt` contains personal information from a fictional website that a user scraped from webserver logs. Within this dataset, there are four fields that interest you:
1. Email Addresses (assume they are alphanumeric user-names and domain-names),
2. [Social Security Numbers](https://en.wikipedia.org/wiki/Social_Security_number#Structure)
3. Bitcoin Addresses (alpha-numeric strings of long length)
4. Street Addresses

Create a function `extract_personal` that takes in a string like `open('data/messy.txt').read()` and returns a tuple of four separate lists containing values of the 4 pieces of information listed above (in the order given). Do **not** keep empty values.

*Hint*: There are multiple "delimiters" in use in the file; there are few enough of them that you can safely determine what they are.

*Note:* Since this data is messy/corrupted, your function will be allowed to miss ~5% of the records in each list. Good spot checking using certain useful substrings (e.g. `@` for emails) should help assure correctness! Your function will be tested on a sample of the file `messy.txt`.

In [262]:
fp = os.path.join('data', 'messy.txt')
s = open(fp, encoding='utf8').read()

In [320]:
emails= re.findall('(\w*@\w*\.\w{1,3})', s)
ssns = re.findall('(\d{3}-\d{2}-\d{4})', s)
bits = re.findall('[\w]*', s)
bits = list(filter(lambda x: len(x)> 20 and len(x) < 50, bits))
adds = re.findall('\d{2,4} [a-zA-Z]+ [a-zA-Z]+', s)

In [265]:
s[:2000]

'1\t4/12/2018\tLorem ipsum dolor sit amet, consectetuer adipiscing elit. Proin risus. Praesent lectus.\n\nVestibulum quam sapien| varius ut, blandit non, interdum in, ante. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Duis faucibus accumsan odio. Curabitur convallis.|dottewell0@gnu.org\toR1mOq,!@#$%^&*(),[{bitcoin:18A8rBU3wvbLTSxMjqrPNc9mvonpA4XMiv\tIP:192.232.9.210\tccn:3563354617955160|ssn:380-09-9403}]|05-6609813,814 Monterey Court\n2\t12/18/2018\tSuspendisse potenti. In eleifend quam a odio. In hac habitasse platea dictumst.\n\nMaecenas ut massa quis augue luctus tincidunt. Nulla mollis molestie lorem. Quisque ut erat.,bassiter1@sphinn.com\tc5KvmarHX3o,test\u2060test\u202b,[{bitcoin:1EB7kYpnfJSqS7kUFpinsmPF3uiH9sfRf1,IP:20.73.13.197|ccn:3542723823957010\tssn:118-12-8276}#{bitcoin:1E5fev4boabWZmXvHGVkHcNJZ2tLnpM6Zv*IP:238.206.212.148\tccn:337941898369615,ssn:427-22-9352}#{bitcoin:1DqG3WcmGw74PjptjzcAmxGFuQdvWL7RCC,IP:171.241.15.98\tccn:3574

## Content in Amazon review data

**Question 3**

The dataset `reviews.txt` contains [Amazon reviews](http://jmcauley.ucsd.edu/data/amazon/) for ~200k phones and phone accessories. This dataset has been "cleaned" for you. The goal of this section is to create a function that takes in the review dataset and a review and returns the word that "best summarizes the review" using TF-IDF.'

1. Create a function `tfidf_data(review, reviews)` that takes a review as well as the review data and returns a dataframe:
    - indexed by the words in `review`,
    - with columns given by (a) the number of times each word is found in the review (`cnt`), (b) the term frequency for each word (`tf`), (c) the inverse document frequency for each word (`idf`), and (d) the TF-IDF for each word (`tfidf`).
    
2. Create a function `relevant_word(tfidf_data)` which takes in a dataframe as above and returns the word that "best summarizes the review" described by `tfidf_data`.


*Note:* Use this function to "cluster" review types -- run it on a sample of reviews and see which words come up most. Unfortunately, you will likely have to change your code from your answer above to run it on the entire dataset (to do this, you should compute as many of the frequencies "ahead of time" and look them up when needed; you should also likely filter out words that occur "rarely")

In [321]:
fp = os.path.join('data', 'reviews.txt')
reviews = pd.read_csv(fp, header=None, squeeze=True)
review = open(os.path.join('data', 'review.txt'), encoding='utf8').read().strip()

In [322]:
review

'this is a great new case design that i have not seen before it has a slim silicone skin that really locks in the phone to cover and protect your phone from spills and such and also a hard polycarbonate outside shell cover to guard it against damage  this case also comes with different interchangeable skins and covers to create multiple color combinations  this is a different kind of case than the usual chunk of plastic  it is innovative and suits the iphone 5 perfectly'

### Tweet Analysis: Internet Research Agency

The dataset `data/ira.csv` contains tweets tagged by Twitter as likely being posted by the *Internet Research Angency* (the tweet factory facing allegations for attempting to influence US political elections).

The questions in this section will focus on the following:
1. We will look at the hashtags present in the text and trends in their makeup.
2. We will prepare this dataset for modeling by creating features out of the text fields.

**Question 4 (HashTags)**

You may assume that a hashtag is any string without whitespace following a `#` (this is more permissive than Twitters rules for hashtags; you are encouraged to go down this rabbit-hole to better figure out how to clean your data!).

* Create a function `hashtag_list` that takes in a column of tweet-text and returns a column containing the list of hashtags present in the tweet text. If a tweet doesn't contain a hashtag, the function should return an empty list.

* Create a function `most_common_hashtag` that takes in a column of hashtag-lists (the output above) and returns a column consisting a single hashtag from the tweet-text. 
    - If the text has no hashtags, the entry should be `NaN`,
    - If the text has one distinct hashtag, the entry should contain that hashtag,
    - If the text has more than one hashtag, the entry should be the most common hashtag (among all hashtags in the column). If there is a tie for most common, any of the most common can be returned.
        - E.g. if the input column was: `pd.Series([[1, 2, 2], [3, 2, 3]])`, the output would be: `pd.Series([2, 2])`. Even though `3` was more common in the second list, `2` is the most common among all hashtags in the column.

In [ ]:
fp = os.path.join('data', 'ira.csv')
ira = pd.read_csv(fp, names=['id', 'name', 'date', 'text'])

**Question 5 (Features)**

Now create a dataframe of features from the `ira` data.  That is create a function `create_features` that takes in the `ira` data and returns a dataframe with the same index as `ira` (i.e. the rows correspond to the same tweets) and the following columns:
* `num_hashtags` gives the number of hashtags present in a tweet,
* `mc_hashtags` gives the most common hashtag associated to a tweet (as given by the problem above),
* `num_tags` gives the number of tags a given tweet has (look for the presence of `@`),
* `num_links` gives the number of hyper-links present in a given tweet 
    - (a hyper-link is a string starting with `http(s)://` not followed by whitespaces),
* A boolean column `is_retweet` that describes if the given tweet is a retweet (i.e. `RT`),
* A 'clean' text field `text` that contains the tweet text with:
    - The non-alphanumeric characters removed (except spaces),
    - All words should be separated by exactly one space,
    - The characters all lowercase,
    - All the meta-information above (Retweet info, tags, hyperlinks, hashtags) removed.

*Note:* You should make a helper function for each column.

*Note:* This will take a while to run on the entire dataset -- test it on a small sample first!

## Congratulations! You're done!

* Submit the lab on Gradescope